<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Emotionline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emitionline 데이터셋

In [1]:
! tar -xvzf /content/drive/My\ Drive/Datasets/Emotionlines/EmotionLines_friends_annotation.tar.gz

EmotionLines/Friends/friends_train.json
EmotionLines/Friends/friends_dev.json
EmotionLines/Friends/friends_test.json
EmotionLines/README


In [2]:
import pandas as pd
import json

In [3]:
with open('/content/EmotionLines/Friends/friends_train.json', encoding='utf-8', mode='r') as f:
  array = json.load(f)

data =[]
for i in range(len(array)):
  data += array[i]
print(f"Total length of data : {len(data)}")
df = pd.DataFrame(data)

Total length of data : 10561


In [4]:
df

,speaker,utterance,emotion,annotation
0,Chandler,also I was the point person on my companys tr...,neutral,4100000
1,The Interviewer,You mustve had your hands full.,neutral,5000000
2,Chandler,That I did. That I did.,neutral,5000000
3,The Interviewer,So lets talk a little bit about your duties.,neutral,5000000
4,Chandler,My duties? All right.,surprise,2000030
...,...,...,...,...
10556,Chandler,You or me?,neutral,3000011
10557,Ross,"I got it. Uh, Joey, women don't have Adam's ap...",non-neutral,2100011
10558,Joey,"You guys are messing with me, right?",surprise,0000050
10559,All,Yeah.,neutral,4000010


In [5]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [6]:
# change the value to dtype: Category
df['emotion'] = pd.Categorical(df['emotion'])

# cat(egory).codes print the code of category
emotion_codes = df.emotion.cat.codes

In [7]:
emotion_codes

0        4
1        4
2        4
3        4
4        7
        ..
10556    4
10557    5
10558    7
10559    4
10560    5
Length: 10561, dtype: int8

In [8]:
df['emotion'].value_counts()

neutral        4752
non-neutral    2017
joy            1283
surprise       1220
anger           513
sadness         351
disgust         240
fear            185
Name: emotion, dtype: int64

In [9]:
for i, utter in enumerate(df.utterance):
  if i < 5:
    print(utter)
  else:
    break

also I was the point person on my companys transition from the KL-5 to GR-6 system.
You mustve had your hands full.
That I did. That I did.
So lets talk a little bit about your duties.
My duties?  All right.


# make data



make the dataset to tensor

In [10]:
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

In [11]:
x_train = df.pop('utterance')
y_train = emotion_codes

In [12]:
x_train.values

array(['also I was the point person on my company\x92s transition from the KL-5 to GR-6 system.',
       'You must\x92ve had your hands full.', 'That I did. That I did.',
       ..., 'You guys are messing with me, right?', 'Yeah.',
       'That was a good one. For a second there, I was like, "whoa."'],
      dtype=object)

In [13]:
y_onehot = tf.one_hot(y_train, depth=8)
y_onehot.shape

TensorShape([10561, 8])

이 이하는 실행 안 해도 됨

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(x_train)

In [ ]:
for t in dataset.take(5):
  print(t)

tf.Tensor(b'also I was the point person on my company\xc2\x92s transition from the KL-5 to GR-6 system.', shape=(), dtype=string)
tf.Tensor(b'You must\xc2\x92ve had your hands full.', shape=(), dtype=string)
tf.Tensor(b'That I did. That I did.', shape=(), dtype=string)
tf.Tensor(b'So let\xc2\x92s talk a little bit about your duties.', shape=(), dtype=string)
tf.Tensor(b'My duties?  All right.', shape=(), dtype=string)


In [ ]:
train_data = dataset.shuffle(len(df))
for t in train_data.take(5):
  print(t)

tf.Tensor(b"Well, I've been reading up and for your information, minks are not very nice.", shape=(), dtype=string)
tf.Tensor(b'Huh, what can we do in 17 minutes?', shape=(), dtype=string)
tf.Tensor(b'But ah, hey, oh, somebody\xc2\x92s off the phone, how \xc2\x91bout a glass of wine by the fire, I could get it going again.', shape=(), dtype=string)
tf.Tensor(b'A TV as if it appears from nowhere! That\xc2\x92s the dream! Man, how did you afford this stuff?', shape=(), dtype=string)
tf.Tensor(b'Yeah, that really calms me down.', shape=(), dtype=string)


# Tokenizer

In [14]:
tokenizer = K.preprocessing.text.Tokenizer(num_words = 4000, char_level=False)

In [15]:
tokenizer.fit_on_texts(x_train.values)

In [16]:
x_in = tokenizer.texts_to_sequences(x_train)

In [17]:
x_in = tf.keras.preprocessing.sequence.pad_sequences(x_in)
print(x_in)

[[   0    0    0 ... 2888  905  906]
 [   0    0    0 ...   40  709  827]
 [   0    0    0 ...    9    1   73]
 ...
 [   0    0    0 ...   36   14   34]
 [   0    0    0 ...    0    0   17]
 [   0    0    0 ...   35   42  177]]


In [18]:
x_in.shape

(10561, 68)

# Embedding

In [25]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, ReLU, Softmax

## Hyperparameter

In [26]:
INPUT_DIM = 4000
EMBED_DIM = 100
INPUT_LEN = 68
FIRST_DENSE = 500
OUTPUT_LEN = 8
BATCH_SIZE = 100
EPOCHS = 10
SHUFFLE = True

Model Create

In [27]:
model = K.Sequential()
model.add(Embedding(input_dim = INPUT_DIM, output_dim=EMBED_DIM, input_length=INPUT_LEN))
model.add(Flatten())
model.add(Dense(FIRST_DENSE))
model.add(ReLU())
model.add(Dense(OUTPUT_LEN))
model.add(Softmax())

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 68, 100)           400000    
_________________________________________________________________
flatten (Flatten)            (None, 6800)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               3400500   
_________________________________________________________________
re_lu (ReLU)                 (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 4008      
_________________________________________________________________
softmax (Softmax)            (None, 8)                 0         
Total params: 3,804,508
Trainable params: 3,804,508
Non-trainable params: 0
____________________________________________

In [28]:
model.compile(optimizer="Adam", loss=K.losses.CategoricalCrossentropy())

In [29]:
model.fit(x=x_in, y=y_onehot, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=SHUFFLE)

Epoch 1/10
106/106 [==============================] - 6s 58ms/step - loss: 1.5826
Epoch 2/10
106/106 [==============================] - 6s 59ms/step - loss: 1.3565
Epoch 3/10
106/106 [==============================] - 6s 58ms/step - loss: 1.0805
Epoch 4/10
106/106 [==============================] - 6s 59ms/step - loss: 0.8018
Epoch 5/10
106/106 [==============================] - 6s 57ms/step - loss: 0.5945
Epoch 6/10
106/106 [==============================] - 6s 58ms/step - loss: 0.4677
Epoch 7/10
106/106 [==============================] - 6s 58ms/step - loss: 0.3954
Epoch 8/10
106/106 [==============================] - 6s 58ms/step - loss: 0.3510
Epoch 9/10
106/106 [==============================] - 6s 58ms/step - loss: 0.3192
Epoch 10/10
106/106 [==============================] - 6s 58ms/step - loss: 0.2984


# Model evaluation

In [45]:
def make_test_sample(text):
  test_data = tokenizer.texts_to_sequences(text)
  test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=68)
  print(test_data)
  print(test_data.shape)
  return test_data

In [46]:
test_data = make_test_sample(["Hello Hello I'm Jisoo"])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0 191 191  52]]
(1, 68)


In [47]:
result = model.predict(test_data, batch_size=BATCH_SIZE)

In [51]:
answer = result.argmax()
print(emo_cat.categories[answer])

neutral


# Categories And codes

In [49]:
emo_cat = pd.Categorical(df['emotion'])
emo_cat.categories
emo_cat.codes

array([4, 4, 4, ..., 7, 4, 5], dtype=int8)

In [50]:
emo_df = pd.DataFrame(
    {"emotion" : emo_cat.categories}
)
emo_df

,emotion
0,anger
1,disgust
2,fear
3,joy
4,neutral
5,non-neutral
6,sadness
7,surprise


'anger'